![title](https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png)

# JWebbinar 38: Photometry with JWST - Imviz
Author: Camilla Pacifici, Space Telescope Science Institute (adapting notebook by Clare Shanahan)

## Tutorial Overview
This tutorial will demonstrate how to use [Imviz](https://jdaviz.readthedocs.io/en/latest/imviz/index.html) to support a photometry workflow.

1. Start Imviz and load data.
2. Setting display options.
3. Aligning images by WCS.
4. Load catalog of sources from the interface and from the notebook.
5. Load footprints.
6. Select a source and do aperture photometry.

The first half of the tutorial will show you how to perform these tasks in the interface. In the second part, we will see how the same tasks can be performed with code in the notebook.

## Import packages

First, import everything we need...

In [14]:
from jdaviz import Imviz
from astropy.table import QTable
from astropy import units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion

## Initialize Imviz

In [17]:
imviz = Imviz()
imviz.show()

Application(config='imviz', docs_link='https://jdaviz.readthedocs.io/en/latest/imviz/index.html', events=['cal…

## Loading Data
In addition to loading local data files or array data (e.g., a Spectrum1D for specviz/specviz2d, or numpy array/CCDData etc in imviz) in a notebook, jdaviz can download and load data directly from the MAST archive when given a URI.

We will download two level 3 NIRCAM images of NGC 346 in two different filters (F335M and F227M) and load them as two layers
in the same image viewer.

(With 'cache=True', a local copy of the data is saved so it will not need to be downloaded next time.)

In [18]:
filenames = ['/Users/cpacifici/DATA/COSMOSWEB/mosaic_nircam_f115w_COSMOS-Web_30mas_v0_2_i2d.fits',
             '/Users/cpacifici/DATA/COSMOSWEB/mosaic_nircam_f277w_COSMOS-Web_60mas_v0_2_i2d.fits',
             '/Users/cpacifici/DATA/COSMOSWEB/mosaic_nircam_f444w_COSMOS-Web_60mas_v0_2_i2d.fits'
            ]

with imviz.batch_load():  # not necessary, but this context manager makes loading multiple files more efficient          
    for filename in filenames:
        # imviz.load_data(f'mast:JWST/product/{filename}', cache=True)"  # to re-download from MAST (or use cached files)
        imviz.load_data(f'{filename}', cache=True)  # pre-downloaded data

Once these files are loaded (which will take longer the first time if they need to be downloaded), they will appear in Imviz in the cell above. Both datasets (given default labels 'A', 'B', and 'C') were loaded into the same viewer. They appear in the data menu on the left hand side. You can select/deselect loaded data to display, remove/re-add data from the viewer, and delete loaded data from the app. To blink between images, press the 'b' key (note that blinking will de-select non active layers).

### Linking by WCS

By default, images are pixel linked when loaded. We can link them by WCS in the 'Orientation' plugin. (This doesn't matter so much in this case, it so happens, but we want to be using world coordinates for catalog functionality).

In [ ]:
plg_orient = imviz.plugins['Orientation']
plg_orient.align_by = "WCS"

## Modifying Image Display Options
Now, we will modify some of the display options to better suit our data. For the live demo, we will do this in the UI in the 'Plot Options' plugin by modifying the image stretch from linear to logarithmic, and setting vmax to a more appropriate value. We will make use of 'layer multiselect' to apply these options to all images at the same time, but you can set different display options for each image independently as well. The following cell accomplishes the same task from the API. 

(Take a look around the Plot Options plugin, there is a lot more you can do to customize display settings including colormap, setting layer colors and opacities to create composite RGB images, and displaying contours.)

In [ ]:
plg_plot = imviz.plugins['Plot Options']

print(plg_plot.stretch_function.choices)
print(plg_plot.stretch_function.value)
print()
print(plg_plot.image_colormap.choices)
print(plg_plot.image_colormap.value)

In [ ]:
# the following code is the API equivalent to the series of UI clicks we will do in the live demo

# get the 'Plot Options' plugin
plg_plot = imviz.plugins['Plot Options']

# enable mutiselect so our chosen options are applied to all images
plg_plot.layer.multiselect = True
plg_plot.select_all()

# switch stretch function from default linear to log
plg_plot.stretch_function = 'arcsinh'

# use the 99.5% stretch function preset
plg_plot.stretch_preset = '99.5%'

# increase vmax to a more suitable value
plg_plot.stretch_vmax = 1
plg_plot.stretch_vmin = 0


Now that we know how to set our own plot options, let's use one of the RGB presets, just for fun. This will apply preset color, stretch, and opacity settings to each layer.

In [ ]:
plg_plot = imviz.plugins['Plot Options']
plg_plot.image_color_mode = 'Color'
plg_plot.apply_RGB_presets()

## Loading Catalogs

SDSS and Gaia catalogs can be loaded directly from jdaviz (with more catalog support planned in the future). Additionally, you can load your own catalog into the application.

In this demo, we are going to query for Gaia sources in the FOV of our image, plot some of them over the image, select them all and zoom to the region containing the sources using 'zoom_to_selected', and finally we will select just one source and zoom to that.

In [4]:
temp_cat = 'temp_catalog.ecsv'
cat = QTable.read(temp_cat)
cat

label,sky_centroid
,"deg,deg"
int64,SkyCoord
1,"149.95284346306025,2.386254387160591"
2,"149.95551514121357,2.3850644734585216"
3,"149.95248259026036,2.3901711840984894"
4,"149.95002290207427,2.3915991548887416"
5,"149.94436169789967,2.391923121958097"
6,"149.94294106006635,2.392312479130075"
7,"149.94113964301883,2.3852143581160914"
8,"149.9532408923092,2.3966809089874204"


In [5]:
plg_cat = imviz.plugins['Catalog Search']
plg_cat.import_catalog('temp_catalog.ecsv')

## Load footprints

In [13]:
plg_fprint = imviz.plugins['Footprints']
plg_fprint.add_overlay('nirspec')
#plg_fprint.overlay = 'nirspec'
plg_fprint.preset = 'NIRSpec'
plg_fprint.ra = cat['sky_centroid'][0].ra.value
plg_fprint.dec = cat['sky_centroid'][0].dec.value
plg_fprint.pa = 30

## Aperture photometry

Lets do some rough analysis of our single selected gaia source. We can draw a circular 'subset' near the souce, then use 'recenter' to centroid the position a little better. Again, we will be doing this in the UI during the demo but the following API calls will replicate these steps.

### Creating and loading regions

In [20]:
# get the 'Subsets Tools' plugin where we can create and interact with spatial regions in imviz
plg_subset = imviz.plugins['Subset Tools']

# load a circular region at the location of our selected catalog item
# just for demo sake, shift the coordinates a tiny bit from the catalog position so we
# can use the 'recenter' position. This recreates the scenario of freehand drawing a circular subset
# rather than placing it at an exact location
circular_region = CircleSkyRegion(center=cat['sky_centroid'][1], radius=3*u.arcsec)
plg_subset.import_region(circular_region)

In [23]:
# use the 'recenter' function to get our drawn region closer to the center of the source
# call this a few times to converge on a better position
plg_subset.subset = 'Subset 1'
plg_subset.recenter()
plg_subset.recenter()
plg_subset.recenter()

### Aperture Photometry
With a subset created and placed on one of the sources in the image, we can use the Aperture Photometry plugin to do some analysis. We can make use of 'batch mode' to get photometry for all loaded images using the same subset (which is useful assuming images are well aligned).

In [24]:
# get the plugin
aperture_photometry = imviz.plugins['Aperture Photometry']._obj

# enable multiselect mode to do photometry on multiple datasets at once
aperture_photometry.multiselect = True

# select all datasets
aperture_photometry.dataset.select_all()

# select our photometric aperture
aperture_photometry.aperture.selected = 'Subset 1'

# and run photometry to produce output table
aperture_photometry.vue_do_aper_phot()

We can look at the output table to see the photometry results for the aperture on each image layer and compare, for example, the magnitude in each filter.

In [25]:
aperture_photometry.table

Table(events=['clear_table', 'popout'], headers_avail=['xcenter', 'ycenter', 'sky_center', 'sum', 'sum_aper_ar…

(Bonus exercise if there is time remaining: Disable multiselect in Aperture Photometry and calculate and display a radial profile on each layer)